# Conversation with CodeAct Agent

CodeAct agent is an agent that not only chats but also writes and executes Python code for you.
More details can be found in the project's related [github repo](https://github.com/xingyaoww/code-act). 

In the following CodeAct agent example, we demonstrate another method of empowering the agent with the capability to invoke tools, specifically by directly providing the agent with the corresponding code for the tools and then allowing the agent to utilize them independently.
Interm of tool usage, ReAct agent also enables the agent to use tools to solve problems, but in a different way. You can refer to the [ReAct agent](https://github.com/modelscope/agentscope/blob/main/examples/conversation_with_react_agent/main.ipynb) example for more details and compare the two.

## Prerequisites

- Follow [READMD.md](https://github.com/modelscope/agentscope) to install AgentScope. We require the lastest version, so you should build from source by running `pip install -e .` instead of intalling from pypi. 
- Prepare a model configuration. AgentScope supports both local deployed model services (CPU or GPU) and third-party services. More details and example model configurations please refer to our [tutorial](https://modelscope.github.io/agentscope/en/tutorial/203-model.html).

## Note
- The example is tested with the following models. For other models, you may need to adjust the prompt.
    - qwen-max

In [ ]:
YOUR_MODEL_CONFIGURATION_NAME = "{YOUR_MODEL_CONFIGURATION_NAME}"

YOUR_MODEL_CONFIGURATION = {
    "model_type": "xxx", 
    "config_name": YOUR_MODEL_CONFIGURATION_NAME
    
    # ...
}

### Step 1: Initalize the AgentScope environment

Here we initialize the AgentScope environment by calling the `agentscope.init` function. The `model_configs` parameter specifies the path to the model configuration file.

In [ ]:
import agentscope

agentscope.init(model_configs=YOUR_MODEL_CONFIGURATION)

### Step 2: Set up the code for tool usage, and init the CodeAct Agent.

Here, we provide the CodeAct agent with the interactive code that the agent can use during the conversation.

In [ ]:

from loguru import logger
from codeact_agent import CodeActAgent

from agentscope.message import Msg

YOUR_BING_API_KEY = "xxx" # fill in your bing api key here

example_code = f"""
from agentscope.service import bing_search
import os

api_key = "{YOUR_BING_API_KEY}"

def search(question: str):
    return bing_search(question, api_key=api_key, num_results=3).content
"""

import nest_asyncio
nest_asyncio.apply()
agent = CodeActAgent(
    name="assistant",
    model_config_name=YOUR_MODEL_CONFIGURATION_NAME,
    example_code=example_code
)



### Step 3: Ask the CodeAct-agent to execute tasks

Here, we ask the CodeAct-agent with the example question in the ReAct Paper: `"Aside from the Apple Remote, what other device can control the program Apple Remote was originally designed to interact with?"` as follows, same as in the [ReAct-agent]((https://github.com/modelscope/agentscope/blob/main/examples/conversation_with_react_agent/main.ipynb)) example.

In [2]:
mss = Msg(
    name="user", 
    content="Aside from the Apple Remote, what other device can control the program Apple Remote was originally designed to interact with?", 
    role="user"
)
logger.chat(mss)
answer_mss1 = agent(mss)

2024-05-17 10:58:04.214 | INFO     | agentscope.models.model:__init__:201 - Initialize model by configuration [dashscope_chat-qwen-max]
2024-05-17 10:58:04.215 | INFO     | agentscope.utils.monitor:register:417 - Register metric [qwen-max.call_counter] to SqliteMonitor with unit [times] and quota [None]
2024-05-17 10:58:04.218 | INFO     | agentscope.utils.monitor:register:417 - Register metric [qwen-max.prompt_tokens] to SqliteMonitor with unit [token] and quota [None]
2024-05-17 10:58:04.220 | INFO     | agentscope.utils.monitor:register:417 - Register metric [qwen-max.completion_tokens] to SqliteMonitor with unit [token] and quota [None]
2024-05-17 10:58:04.222 | INFO     | agentscope.utils.monitor:register:417 - Register metric [qwen-max.total_tokens] to SqliteMonitor with unit [token] and quota [None]
user: Example Code excuted: Excution Successful:
 Execution Output:
[]
user: Aside from the Apple Remote, what other device can control the program Apple Remote was originally design

The CodeAct agent successfully use the given search tool functions and return the answer accordingly. This demonstrates the wide range of usage of agent with coding abilities.
Feel free to explore the Agent by yourself!